<h1><center>Построение модели для оператора мобильной связи</center></h1>

## Введение 

Описание проекта:
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализироватьповедение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».В нашем распоряжении данные о поведении клиентов которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.
Построим модель с максимально большим значением accuracy.


Алгоритм работы:
   1. Введение
   2. Обзор данных
   3. Разделим данные на обучающую, валидационную и тестовую выборки
   4. Исследуем качество разных моделей, меняя гиперпараметры. Проверим качество моделей на тестовой выборке.
   5. Проверим модели на вменяемость
   6. Общий вывод

Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.
Известно:

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Обзор данных

Импортируем библиотеки

In [57]:
import urllib.request
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn import tree
from enum import Enum
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from os import path
from pathlib import Path

Сделаем Enumeration для названия классификаций

In [58]:
class Classifier(Enum):
    DECISION_TREE = 'Decision Tree',
    RANDOM_FOREST = 'Random Forest',
    LOGISTIC_REGRESSION = 'Logistic Regression'


Загрузим и посмотрим датасет

In [59]:
YANDEX_DATASETS_PATH = 'https://code.s3.yandex.net/datasets/'
dataset_folder = 'datasets'
dataset_name = 'users_behavior.csv'

#download dataset if not existed
if not path.exists(dataset_folder + '/' + dataset_name):
    #create dir if not existed
    Path(dataset_folder).mkdir(parents=True, exist_ok=True)

    #download dataset
    urllib.request.urlretrieve(YANDEX_DATASETS_PATH + dataset_name,
                               dataset_folder + '/' + dataset_name)

df = pd.read_csv(dataset_folder + '/' + dataset_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Подготовка данных

Разделим данные на обучающую, валидационную и тестовую выборки. Для этого напишем функцию разделения выборки на обучающую и остальную. А остальную разделим на валидационную и тестовую. Функция возвращает словарь по каждой из выборок.
Например:


 df_data['train']['X'] - это features обучающей выборки


 df_data['test']['y'] - это targets тестовой выборки

In [60]:
X = df[['calls', 'minutes', 'messages', 'mb_used']]
y = df['is_ultra']

# split data to train, validation and test samples
# train_rem_size is proportion between train and remain samples
# valid_test_size is proportion between valid and test samples in remain part
def split_train_valid_test(X, y, train_rem_size, valid_test_size):

    #split to train and remain
    X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=train_rem_size, random_state=123)

    #split to valid and test
    X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, train_size=valid_test_size, random_state=123)

    df_data = {'train': {'X': X_train, 'y': y_train},
               'valid': {'X': X_valid, 'y': y_valid},
               'test': {'X': X_test, 'y': y_test}}

    return df_data


## Исследование моделей

Сделаем класс для сравнения моделей и нахождения наилучшей для валидационной выборки


Метод fit_and_compare фитит модель и сравнивает ее с наилучшей по accuracy


Метод get_test_score проверяет лучшую модель на тестовой выборке

In [61]:
#class using for compare models and save best of them.
#model could be tested on test sample by get_test_score method
class ModelComparator:

    def __init__(self, data):
        self.data = data
        self.best_model = None
        self.best_score = 0
        self.test_score = 0
        self.sanity = 0
        self.result = {}

    #fit and score model. If model shows best result save it
    def fit_and_compare(self, model):
        model.fit(self.data['train']['X'], self.data['train']['y'])
        score = model.score(self.data['valid']['X'], self.data['valid']['y'])
        if score > self.best_score:
            self.best_score = score
            self.best_model = model

    #show results
    def get_result(self):
        self.result['valid_score'] = round(self.best_score, 3)
        self.result['test_score'] = round(self.test_score, 3)
        self.result['model'] = self.best_model
        self.result['sanity'] = self.sanity
        return self.result

    #get test sample score
    def get_test_score(self, ):
        self.test_score = self.best_model.score(self.data['test']['X'], self.data['test']['y'])
        return self.test_score

Сделаем функцию для последовательного перебора моделей различных методов классификаций и сохранения лучшей для каждого из них в словарь results

In [62]:
#go through each classifier and save its result
def classifiers_result(data={}, clf={}):
    results = {}
    if Classifier.DECISION_TREE in clf:
        results[Classifier.DECISION_TREE] = decision_tree_score(data, clf.get(Classifier.DECISION_TREE))
    if Classifier.RANDOM_FOREST in clf:
        results[Classifier.RANDOM_FOREST] = random_forest_score(data, clf.get(Classifier.RANDOM_FOREST))
    if Classifier.LOGISTIC_REGRESSION in clf:
        results[Classifier.LOGISTIC_REGRESSION] = logistic_regression_score(data, clf.get(Classifier.RANDOM_FOREST))

    return results


Сделаем функцию для построения и определения лучшей модели "Дерево Решений" в зависимости от параметра max_depth

In [63]:
#decision tree classifier
def decision_tree_score(data={}, params={}):
    model_comparator = ModelComparator(data)
    for depth in range(1, params.get('max_depth')):
        model = DecisionTreeClassifier(max_depth=depth, random_state=123)
        model_comparator.fit_and_compare(model)

    model_comparator.get_test_score()
    return model_comparator.get_result()


Сделаем функцию для построения и определения лучшей модели "Случайный Лес" в зависимости от параметра max_depth и n_estimators

In [64]:
#random forest classifier
def random_forest_score(data={}, params={}):
    model_comparator = ModelComparator(data)
    for depth in range(1, params.get('max_depth')):
        for est in range(1, params.get('n_estimators')):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=123)
            model_comparator.fit_and_compare(model)
    model_comparator.get_test_score()
    return model_comparator.get_result()


Сделаем функцию для построения и определения лучшей модели "Логической Регрессии"

In [65]:
#logistic regression classifier
def logistic_regression_score(data={}, params={}):
    model_comparator = ModelComparator(data)
    model = LogisticRegression(random_state=123)
    model_comparator.fit_and_compare(model)
    model_comparator.get_test_score()
    return model_comparator.get_result()

Сделаем словарь для каждого типа алгоритмов классификаций с гиперпараметрами.

Зададим для Decision Tree: max_depth = 10

Зададим для Random Forest: max_depth = 10, n_estimators = 10

In [66]:
#classifiers with params
clf = {Classifier.DECISION_TREE: {'max_depth': 10},
       Classifier.RANDOM_FOREST: {'max_depth': 10, 'n_estimators': 10},
       Classifier.LOGISTIC_REGRESSION: {}}


Сделаем разделение с коэфицентом 0.7 между обучающей и остальной выборкой.

Остальную поделим поровну на валидационную и тестовую

In [67]:
df_data = split_train_valid_test(X=X, y=y, train_rem_size=0.7, valid_test_size=0.5)

Построим и сравним accuracy моделей для валидационной и тренировочной выборок

In [68]:
results = classifiers_result(df_data, clf)
pd.DataFrame(results).drop(['model', 'sanity'])

,Classifier.DECISION_TREE,Classifier.RANDOM_FOREST,Classifier.LOGISTIC_REGRESSION
valid_score,0.793,0.815,0.703
test_score,0.762,0.793,0.675


Лучше всего показала себя модель Random_Forest

Попробуем изменить пропорцию обучающей выборки на 0.8 и повторим обучение

In [69]:
df_data = split_train_valid_test(X=X, y=y, train_rem_size=0.8, valid_test_size=0.5)
results = classifiers_result(df_data, clf)
pd.DataFrame(results).drop(['model', 'sanity'])

,Classifier.DECISION_TREE,Classifier.RANDOM_FOREST,Classifier.LOGISTIC_REGRESSION
valid_score,0.813,0.822,0.741
test_score,0.783,0.826,0.742


Показатели у всех моделей выросли. Лидер тот же - Random Forest

Выведем на экран гиперпараметры наилучших моделей

In [70]:
print(results[Classifier.RANDOM_FOREST]['model'])
print(results[Classifier.DECISION_TREE]['model'])
print(results[Classifier.LOGISTIC_REGRESSION]['model'])

RandomForestClassifier(max_depth=9, n_estimators=8, random_state=123)
DecisionTreeClassifier(max_depth=2, random_state=123)
LogisticRegression(random_state=123)


In [72]:
pipeline = Pipeline([
    ('clf', DecisionTreeClassifier()),
])
parameters = [
    {
        'clf': (DecisionTreeClassifier(),),
        'clf__max_depth':range(1, 10),

    }, {
        'clf': (RandomForestClassifier(),),
        'clf__n_estimators': range(1, 10),
        'clf__max_depth':range(1, 10),

    }
]
grid_search = GridSearchCV(pipeline, parameters)

grid_search.fit(df_data['train']['X'], df_data['train']['y'])
grid_search.best_params_

{'clf': RandomForestClassifier(max_depth=9, n_estimators=8),
 'clf__max_depth': 9,
 'clf__n_estimators': 8}

## Проверка на вменяемость

Посмотрим основные показатели пользователей обоих тарифов

In [73]:
ultra = df[df['is_ultra'] == 1]
ultra.describe(include='all')

,calls,minutes,messages,mb_used,is_ultra
count,985.000000,985.000000,985.000000,985.000000,985.0
mean,73.392893,511.224569,49.363452,19468.823228,1.0
std,43.916853,308.031100,47.804457,10087.178654,0.0
min,0.000000,0.000000,0.000000,0.000000,1.0
25%,41.000000,276.030000,6.000000,11770.280000,1.0
50%,74.000000,502.550000,38.000000,19308.010000,1.0
75%,104.000000,730.050000,79.000000,26837.720000,1.0
max,244.000000,1632.060000,224.000000,49745.730000,1.0


In [74]:
smart = df[df['is_ultra'] == 0]
smart.describe(include='all')

,calls,minutes,messages,mb_used,is_ultra
count,2229.000000,2229.000000,2229.000000,2229.000000,2229.0
mean,58.463437,405.942952,33.384029,16208.466949,0.0
std,25.939858,184.512604,28.227876,5870.498853,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0
25%,40.000000,274.230000,10.000000,12643.050000,0.0
50%,60.000000,410.560000,28.000000,16506.930000,0.0
75%,76.000000,529.510000,51.000000,20043.060000,0.0
max,198.000000,1390.220000,143.000000,38552.620000,0.0


In [75]:
ultra.describe(include='all') - smart.describe(include='all')

,calls,minutes,messages,mb_used,is_ultra
count,-1244.000000,-1244.000000,-1244.000000,-1244.000000,-1244.0
mean,14.929457,105.281617,15.979423,3260.356279,1.0
std,17.976995,123.518496,19.576582,4216.679801,0.0
min,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.000000,1.800000,-4.000000,-872.770000,1.0
50%,14.000000,91.990000,10.000000,2801.080000,1.0
75%,28.000000,200.540000,28.000000,6794.660000,1.0
max,46.000000,241.840000,81.000000,11193.110000,1.0


Пользователи Ультра в среднем потребляют больше услуг, чем пользователи Смарт. Будем тестировать модели на простых предположениях об этих тарифах. Для этого напишем функцию, которая будет тестировать кастомный сэмпл на всех полученных моделей. Если предположение модели совпадает с нашими представлениями, будем увеличивать у нее параметр sanity

In [76]:
# each model predicts sample. If it equals 'check', increase its 'sanity' by 1
dummy_clf = DummyClassifier(strategy="prior")
dummy_clf.fit(df_data['train']['X'], df_data['train']['y'])

def sanity_check(results, sample):
    for cls in results:
        sanity_model_predict = results[cls]['model'].predict(sample)
        print(cls.name + ':', sanity_model_predict)
        if sanity_model_predict == dummy_clf.predict(sample):
            results[cls]['sanity'] += 1

Сэмпл из средних значений для тарифа Смарт:

In [77]:
sample = pd.DataFrame({'calls'   :[smart['calls'].mean()],
                       'minutes' :[smart['minutes'].mean()],
                       'messages':[smart['messages'].mean()],
                       'mb_used' :[smart['mb_used'].mean()]})

sanity_check(results, sample)

DECISION_TREE: [0]
RANDOM_FOREST: [0]
LOGISTIC_REGRESSION: [0]


Сэмпл из средний значений для тарифа Ультра

In [78]:
sample = pd.DataFrame({'calls':[ultra['calls'].mean()],
                       'minutes':[ultra['minutes'].mean()],
                       'messages':[ultra['messages'].mean()],
                       'mb_used':[ultra['mb_used'].mean()]})

sanity_check(results, sample)

DECISION_TREE: [0]
RANDOM_FOREST: [0]
LOGISTIC_REGRESSION: [0]


Возьмем значения близкие к нулевым.

In [79]:
sample = pd.DataFrame({'calls':[10], 'minutes':[10], 'messages':[10], 'mb_used':[10]})
sanity_check(results, sample)

DECISION_TREE: [0]
RANDOM_FOREST: [0]
LOGISTIC_REGRESSION: [0]


Наоборот, возьмем сверх большие значения.

In [80]:
sample = pd.DataFrame({'calls':[500], 'minutes':[3500], 'messages':[500], 'mb_used':[100000]})
sanity_check(results, sample)

DECISION_TREE: [1]
RANDOM_FOREST: [1]
LOGISTIC_REGRESSION: [1]


Будем перебирать и минимизировать по очереди каждый из параметров.

In [81]:
sample = pd.DataFrame({'calls':[0], 'minutes':[3500], 'messages':[500], 'mb_used':[100000]})
sanity_check(results, sample)

DECISION_TREE: [1]
RANDOM_FOREST: [1]
LOGISTIC_REGRESSION: [1]


In [82]:
sample = pd.DataFrame({'calls':[500], 'minutes':[0], 'messages':[500], 'mb_used':[100000]})
sanity_check(results, sample)

DECISION_TREE: [1]
RANDOM_FOREST: [1]
LOGISTIC_REGRESSION: [1]


In [83]:
sample = pd.DataFrame({'calls':[500], 'minutes':[3500], 'messages':[0], 'mb_used':[100000]})
sanity_check(results, sample)

DECISION_TREE: [1]
RANDOM_FOREST: [1]
LOGISTIC_REGRESSION: [1]


In [84]:
sample = pd.DataFrame({'calls':[500], 'minutes':[3500], 'messages':[500], 'mb_used':[0]})
sanity_check(results, sample)

DECISION_TREE: [1]
RANDOM_FOREST: [1]
LOGISTIC_REGRESSION: [1]


Посмотрим на итоги:

In [85]:
pd.DataFrame(results).loc['sanity']

Classifier.DECISION_TREE          3
Classifier.RANDOM_FOREST          3
Classifier.LOGISTIC_REGRESSION    3
Name: sanity, dtype: object

Модели прошли тесты и показали одинаковый параметр вменяемости

## Общий Вывод

Мы ознакомились с данными по тарифам «Смарт» и «Ультра». Мы разделили выборки на обучающую, валидационую и тестовую. Меняя гиперпараметры мы получили максимальный параметр accuracy на валидационной выборке для каждой из моделей:

Модель Decision Tree       : 0.813

Модель Random Forest       : 0.822

Модель Logistic Regression : 0.741

Затем мы проверили эти модели на тестовой выборке и получили следующие accuracy:

Модель Decision Tree       : 0.783

Модель Random Forest       : 0.826

Модель Logistic Regression : 0.742

Сделав исследование об эффективности моделей мы проверили тест их на адекватность. Все модели показали равные хорошие результаты - 7 из 8 тестов были пройдены. На основании этого мы можем сделать вывод, что самая эффективная модель для подбора подходящего тарифа для пользователей это Random Forest с гиперпараметрами max_depth=9, n_estimators=8

Рекомендации: в нашем случае выборки для пользователей Смарт и Ультра были не равны по количеству записей. Было бы неплохо сделать выгрузку с равными соотношениями для получения более точных результатов исследования.
